In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import os
import seaborn as sns


In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# Read Data
df_mkpf = pd.read_excel('../Data1/MKPF.XLSX')

In [ ]:
df_mkpf.head(4)

In [ ]:
df_mseg = pd.read_excel('../Data1/MSEG.XLSX')

In [ ]:
df_mseg.head(4)

In [ ]:
print(df_mseg.shape)
print(df_mkpf.shape)

In [ ]:
df_mb = pd.merge(df_mseg, df_mkpf, how='left', on='MBLNR')

In [ ]:
#sanity check (first number should be the higher one from before, second should be addition of the right numbers)
df_mb.shape

In [ ]:
df_mb.head(5)

In [ ]:
import missingno as msno
%matplotlib inline
msno.matrix(df_mb)

# Data Cleansing

## initial cleaning

In [ ]:
df = df_mb.copy()

In [ ]:
# Spalten die nur NAN enthalten entfernen
df = df.dropna(axis=1, how='all')

# Spalten die ausschließlich XY Werte enthalten (0 / X / 1 / etc.) dropen
df = df.loc[:, (df.astype(str) != 0).any(axis=0)]
df = df.loc[:, (df.astype(str) != '0,0').any(axis=0)]
df = df.loc[:, (df.astype(str) != '0,00').any(axis=0)]
df = df.loc[:, (df.astype(str) != '0,000').any(axis=0)]
df = df.loc[:, (df.astype(str) != 'X').any(axis=0)]
df = df.loc[:, (df.astype(str) != '1').any(axis=0)]

#spaletn entfernen die immer die selben Werte enthalten
keep_columns = [col for col in df.columns if len(df[col].unique()) > 1]
df = df[keep_columns].copy()


#print(len(df_io.index))
df.shape

In [ ]:
import missingno as msno
%matplotlib inline
msno.matrix(df)

In [ ]:
df.head(4)

In [ ]:
# have lost somhow pretty much of the data due to the sparcity and empty cols

In [ ]:
#lets add the time and date column
#CPUDT_MKPF 	CPUTM_MKPF
df['CPUDT'] = df_mb['CPUDT_MKPF']

#as Timestamp
#df['TMSTMP'] = pd.to_datetime(df['CPUTM'].astype(str)+' '+df['CPUDT'].astype(str))
df['TMSTMP'] = pd.to_datetime(df['CPUDT'].astype(str) + df['CPUTM'].astype(str) , format='%Y-%m-%d%H:%M:%S')

In [ ]:
df.head(4)

In [ ]:
#sanity check
df.shape

In [ ]:
# manual cleaning plant specifics

In [ ]:
df.drop(['WERKS', 'KOKRS', 'BUKRS', 'PRCTR', 'PPRCTR'],axis=1, inplace=True)

In [ ]:
df.MATBF = df.MATBF.str.slice(3)
df.MATNR = df.MATNR.str.slice(3)

In [ ]:
#2 Del
#ZEILE LINE_ID CPUTM_MKPF
df = df.drop(['CPUTM_MKPF'], axis=1)
df = df.drop(['ZEILE'], axis=1)
df = df.drop(['LINE_ID'], axis=1)


In [ ]:
df = df.drop(['XBLNR_MKPF'], axis=1)
df = df.drop(['TCODE2_MKPF'], axis=1)

## data types correction

In [ ]:
df.describe(include='all').T

In [ ]:
df.sample(10)

In [ ]:
cont_Cols = ['MBLNR', ]

#to many we create this list later by computation
cat_Cols = [ ]

num_Cols = ['DMBTR', 'BUALT', 'MENGE', 'ERFMG', 'BPMNG', 'LBKUM', 'SALK3', 'BSTMG'] #LSMNG

dat_Cols = ['CPUTM', 'CPUDT', 'TMSTMP', 'BUDAT', 'BLDAT', 'CPUDT_MKPF', 'BUDAT_MKPF']



In [ ]:
list1 = df.columns.to_list()
print(list1)

#we already have these cols in num or cont or dat so lets make a list and remove them from all other than we have cat cols
unw = num_Cols + dat_Cols + cont_Cols

#cat cols are all cols - the cols we already defined as num / dat / cat
cat_Cols = [x for x in list1 if x not in unw]

In [ ]:
print(cat_Cols)

In [ ]:
#sanity check of cols
print(len(cont_Cols)+len(cat_Cols)+len(num_Cols)+len(dat_Cols))
print(df.shape[1]) 

In [ ]:
# Change dtype to Category for relevant features
df_cor = df.copy()
df_cor.head()

# we have to change NaN top 0 to avoid the infer dtype "object" for some cols
#df_cor['ZUONR'] = df_cor['ZUONR'].fillna(0)


# change Dtype to Cat for categorical columns
cats_cat = df_cor[cat_Cols]
        
for (columnName, columnData) in cats_cat.items():
    df_cor[columnName] = df_cor[columnName].astype(str)
    if isinstance(df_cor[columnName][0], str):
        s = pd.Series(df_cor[columnName], dtype="category")
        df_cor[columnName] = df_cor[columnName].astype('category')



In [ ]:
#check 1
df_cor.dtypes

In [ ]:
#check 2
print('Categorical columns: {}'.format(list(df_cor.select_dtypes('category').columns)))

In [ ]:
#check 3
df_cor.shape

In [ ]:
import missingno as msno
%matplotlib inline
msno.matrix(df_cor)

In [ ]:
df_cor.to_feather('../Data/D3_MKPF-MSEG_raw.ftr')

# lets check the dataset properities / Correlation  / distribution

In [ ]:
# Compute the corr matrix, only for numerics since we use category dtype
corr = df_cor.corr(numeric_only=True).round(2)

# Set up the triangular mask / matplotlib figure / Colors
mask = np.triu(np.ones_like(corr, dtype=bool)) # tril for upper
f, ax = plt.subplots(figsize=(7, 7))
cmap1 = sns.cubehelix_palette(dark=0, light=1, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap1, vmax=1,vmin=-1, center=0, square=True, annot=True, linewidths=.5, cbar_kws={"shrink": .5})


In [ ]:

# Compute the corr matrix and 
corr = df.corr(numeric_only=True).round(2)

# Set up the triangular mask / matplotlib figure / Colors
mask = np.triu(np.ones_like(corr, dtype=bool)) # tril for upper
f, ax = plt.subplots(figsize=(20, 20))
cmap1 = sns.cubehelix_palette(dark=0, light=1, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap1, vmax=1,vmin=-1, center=0, square=True, annot=True, linewidths=.5, cbar_kws={"shrink": .5})


In [ ]:
df.sample(10)

In [ ]:
#manually delete the hihgly correlating features in both dataframes
# WRBTR PSWBT VBEL2 POSN2 ERFMG AUFPL SAKNR SAKNR LOKKT KIDNO
df_cor = df_cor.drop(['ERFMG'], axis=1)
df = df.drop(['ERFMG'], axis=1)

df_cor = df_cor.drop(['WEMPF'], axis=1)
df = df.drop(['WEMPF'], axis=1)

df_cor = df_cor.drop(['AUFPL'], axis=1)
df = df.drop(['AUFPL'], axis=1)



In [ ]:
df_cor = df_cor.drop(['VBELP_IM'], axis=1)
df = df.drop(['VBELP_IM'], axis=1)

df_cor = df_cor.drop(['VBELN_IM'], axis=1)
df = df.drop(['VBELN_IM'], axis=1)

In [ ]:
df_cor = df_cor.drop(['XBLNR'], axis=1)
df = df.drop(['XBLNR'], axis=1)

df_cor = df_cor.drop(['LE_VBELN'], axis=1)
df = df.drop(['LE_VBELN'], axis=1)

df_cor = df_cor.drop(['WEVER'], axis=1)
df = df.drop(['WEVER'], axis=1)


In [ ]:
# Compute the corr matrix and 
corr = df.corr(numeric_only=True).round(2)

# Set up the triangular mask / matplotlib figure / Colors
mask = np.triu(np.ones_like(corr, dtype=bool)) # tril for upper
f, ax = plt.subplots(figsize=(20, 20))
cmap1 = sns.cubehelix_palette(dark=0, light=1, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap1, vmax=1,vmin=-1, center=0, square=True, annot=True, linewidths=.5, cbar_kws={"shrink": .5})


In [ ]:
df_cor.head()

In [ ]:
df_cor.to_feather('../Data/D3_MKPF-MSEG_cleaned.ftr')